# Overview

 - Imprt main libraries
 - Load and Save Files
 - Running Simulations
 - MFI algorithms
 - Integraiton in 1D
 - Integration in 2D
 - Calculate Error
 - Other Functions

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import statistics
import scipy.integrate as integrate
import plumed
import pandas as pd
from labellines import labelLines
from labellines import labelLine
import scipy.io
import matplotlib as mpl

from matplotlib import rc
plt.rcParams.update({ "text.usetex": True, "font.family": "serif", "font.serif": ["computer modern roman"], "font.size": 14})
plw = 0.6
pcs = 3
pms = 3
bfillc = [0.9,0.9,0.9]
plt.rcParams['axes.linewidth'] = plw
plt.rcParams['xtick.top'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['xtick.major.width'] = plw
plt.rcParams['xtick.minor.width'] = plw
plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['xtick.major.size'] = 4.5
plt.rcParams['ytick.right'] = True
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['ytick.major.width'] = plw
plt.rcParams['ytick.minor.width'] = plw
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.major.size'] = 5
plt.rcParams["figure.figsize"] = (5,4)

# Load and Save Files

In [6]:
#Load the HILLS data
def load_HILLS_1D(hills_name = "HILLS"):
    for file in glob.glob(hills_name):
        hills = np.loadtxt(file)
        hills = np.concatenate(([hills[0]], hills[:-1]))
        hills[0][3] = 0
    return hills

#Load the trajectory (position) data
def laod_position_1D(position_name = "position"):
    for file1 in glob.glob(position_name):
        colvar = np.loadtxt(file1)
    return colvar[:-1, 1]

def laod_positiontime_1D(position_name = "position"):
    for file1 in glob.glob(position_name):
        colvar = np.loadtxt(file1)
        time = colvar[:-1, 0]
    return time

def load_HILLS_2D(hills_name = "HILLS"):
    for file in glob.glob(hills_name):
        hills = np.loadtxt(file)
        hills = np.concatenate(([hills[0]], hills[:-1]))
        hills[0][5] = 0
    return hills

def laod_position_2D(position_name = "position"):
    for file1 in glob.glob(position_name):
        colvar = np.loadtxt(file1)
        position_x = colvar[:-1, 1]
        position_y = colvar[:-1, 2]
    return [position_x, position_y]

##################################################################

def save_pkl(object, file_name):
    with open(file_name, "wb") as fw:
        pickle.dump(object, fw, pickle.HIGHEST_PROTOCOL)

def load_pkl(name):
    with open(name, "rb") as fr:
        return pickle.load(fr)

def save_npy(object, file_name):
    with open(file_name, "wb") as fw:
        np.save(fw, object)

def load_npy(name):
    with open(name, "rb") as fr:
        return np.load(fr)

# Runing Simulations

In [3]:
### Run a 1D Langevin simulation using the analytical force field y=7*x^4-23*x^2
### Produces a position file that contains the CV value over time, a HILLS
### file that contains informaton on the history dependet bias, a histo file that 
### contains the histogram of the CV value and a fes.dat file that contains the FES.

def run_langevin1D_plumed_fes(length, fes_stride=0, sigma=0.1, height=0.1, biasfactor=10):
    with open("plumed.dat","w") as f:
        print("""#Define system as distance between two atoms
p: DISTANCE ATOMS=1,2 COMPONENTS
#Define Force field
ff: MATHEVAL ARG=p.x PERIODIC=NO FUNC=(7*x^4-23*x^2)
bb: BIASVALUE ARG=ff
#Define Histroy dependet bias potentail (baisfactor very large so that height stays constant)
# METAD ARG=p.x PACE=100 SIGMA=0.1 HEIGHT=0.1 TEMP=120 BIASFACTOR=10000
METAD ARG=p.x PACE=100 SIGMA={} HEIGHT={} GRID_MIN=-3 GRID_MAX=3 GRID_BIN=600 BIASFACTOR={} TEMP=120 CALC_RCT
#Reweight Bias
bias: REWEIGHT_METAD TEMP=120
#Make Histogram
hh: HISTOGRAM ARG=p.x GRID_MIN=-3 GRID_MAX=3 GRID_BIN=600 BANDWIDTH=0.01 LOGWEIGHTS=bias
#Convert Histogram to FES
fes: CONVERT_TO_FES GRID=hh TEMP=120
#Save Histogram and FES at the end. Save position every 10 time-steps    
#DUMPGRID GRID=hh FILE=histo STRIDE={}
DUMPGRID GRID=fes FILE=fes.dat STRIDE={}
PRINT FILE=position ARG=p.x STRIDE=10""".format(sigma, height, biasfactor, fes_stride, fes_stride),file=f)

    with open("input","w") as f:
        print("""temperature 1
tstep 0.005
friction 1
dimension 1
nstep {}
ipos -1.0
periodic false""".format(length),file=f)

    #Start WT-Metadynamic simulation
    !plumed pesmd < input
    
    
#Runnig simulation with the intention to alanyse the results with MFI works the same  
#way as for metadynamcis and WT-metadynamics, but one doesnt need to do the reweighting  
#or calculate the histogram, just save the trajectory and the HILLS. 

def run_langevin1D(length, sigma=0.1, height=0.1, biasfactor=10):
    with open("plumed.dat","w") as f:
        print("""p: DISTANCE ATOMS=1,2 COMPONENTS
ff: MATHEVAL ARG=p.x PERIODIC=NO FUNC=(7*x^4-23*x^2)
bb: BIASVALUE ARG=ff
METAD ARG=p.x PACE=100 SIGMA={} HEIGHT={} GRID_MIN=-3 GRID_MAX=3 GRID_BIN=200 BIASFACTOR={} TEMP=120
PRINT FILE=position ARG=p.x STRIDE=10""".format(sigma,height, biasfactor),file=f)

    with open("input","w") as f:
        print("""temperature 1
tstep 0.005
friction 1
dimension 1
nstep {}
ipos -1.0
periodic false""".format(length),file=f)
    
    !plumed pesmd < input
    
    
#Run Metadynamics with umbrella sampling in 1D
def run_langevin1D_hp(sim_len, kappa, hp_pos, sigma = 0.1, height = 1, biasfactor = 10):
    with open("plumed.dat","w") as f:
        print("""p: DISTANCE ATOMS=1,2 COMPONENTS
ff: MATHEVAL ARG=p.x PERIODIC=NO FUNC=(7*x^4-23*x^2)
bb: BIASVALUE ARG=ff
RESTRAINT ARG=p.x AT={} KAPPA={} LABEL=restraint
METAD ARG=p.x PACE=100 SIGMA={} HEIGHT={} GRID_MIN=-3 GRID_MAX=3 GRID_BIN=300 BIASFACTOR={} TEMP=120 CALC_RCT
PRINT FILE=position ARG=p.x STRIDE=10
""".format(hp_pos,kappa, sigma, height, biasfactor),file=f)

    with open("input","w") as f:
        print("""temperature 1
tstep 0.005
friction 1
dimension 1
nstep {}
ipos {}
periodic false""".format(sim_len, hp_pos),file=f)
        
    #Start simulation
    !plumed pesmd < input
    
############################################################################################

### Run a 2D Langevin simulation using the analytical force field z=7*x^4-23*x^2 + 7*y^4-23*y^2  
def run_langevin2D_plumed_fes(length, sigma=0.1, height=0.1, biasfactor=10):
    with open("plumed.dat","w") as f:
        print("""p: DISTANCE ATOMS=1,2 COMPONENTS
ff: MATHEVAL ARG=p.x,p.y PERIODIC=NO FUNC=(7*x^4-23*x^2+7*y^4-23*y^2)
bb: BIASVALUE ARG=ff
METAD ARG=p.x,p.y PACE=100 SIGMA={},{} HEIGHT={} GRID_MIN=-3,-3 GRID_MAX=3,3 GRID_BIN=300,300 BIASFACTOR={} TEMP=120 CALC_RCT
bias: REWEIGHT_METAD TEMP=120
hh: HISTOGRAM ARG=p.x,p.y GRID_MIN=-3,-3 GRID_MAX=3,3 GRID_BIN=300,300 BANDWIDTH=0.02,0.02 LOGWEIGHTS=bias
fes: CONVERT_TO_FES GRID=hh TEMP=120
DUMPGRID GRID=fes FILE=fes.dat STRIDE={}
PRINT FILE=position ARG=p.x,p.y STRIDE=10""".format(sigma, sigma, height, biasfactor, length),file=f)

    with open("input","w") as f:
        print("""temperature 1
tstep 0.005
friction 1
dimension 2
nstep {}
ipos -1.0,-1.0
periodic false""".format(length),file=f)

    !plumed pesmd < input 
    
    
### Run a 2D Langevin simulation using the analytical force field z=7*x^4-23*x^2 + 7*y^4-23*y^2
def run_langevin2D(length, sigma=0.1, height=0.1, biasfactor=10):
    with open("plumed.dat","w") as f:
        print("""p: DISTANCE ATOMS=1,2 COMPONENTS
ff: MATHEVAL ARG=p.x,p.y PERIODIC=NO FUNC=(7*x^4-23*x^2+7*y^4-23*y^2)
bb: BIASVALUE ARG=ff
METAD ARG=p.x,p.y PACE=100 SIGMA={},{} HEIGHT={} GRID_MIN=-3,-3 GRID_MAX=3,3 GRID_BIN=300,300 BIASFACTOR={} TEMP=120 CALC_RCT
PRINT FILE=position ARG=p.x,p.y STRIDE=10""".format(sigma, sigma, height, biasfactor, length),file=f)

    with open("input","w") as f:
        print("""temperature 1
tstep 0.005
friction 1
dimension 2
nstep {}
ipos -1.0,-1.0
periodic false""".format(length),file=f)

    !plumed pesmd < input 
    
    
#Run Metadynamics with umbrella sampling in 2D    
def run_langevin2D_hp(length, kappa, ipos_x, ipos_y, sigma = 0.1, height = 1, biasfactor = 10):
    with open("plumed.dat","w") as f:
        print("""p: DISTANCE ATOMS=1,2 COMPONENTS
ff: MATHEVAL ARG=p.x,p.y PERIODIC=NO FUNC=(7*x^4-23*x^2+7*y^4-23*y^2)
bb: BIASVALUE ARG=ff
#Define Harmonic potential
RESTRAINT ARG=p.x,p.y AT={},{} KAPPA={},{} LABEL=restraint
METAD ARG=p.x,p.y PACE=100 SIGMA={},{} HEIGHT={} GRID_MIN=-3,-3 GRID_MAX=3,3 GRID_BIN=300,300 BIASFACTOR={} TEMP=120 CALC_RCT
PRINT FILE=position ARG=p.x,p.y STRIDE=10""".format(ipos_x,ipos_y,kappa,kappa, sigma, sigma, height, biasfactor),file=f)

    with open("input","w") as f:
        print("""temperature 1
tstep 0.005
friction 1
dimension 2
nstep {}
ipos 0.0,0.0
periodic false""".format(length),file=f)
        
    !plumed pesmd < input

# MFI algorithms

In [44]:
### Algorithm to run 1D MFI

def MFI_1D_simple():
    #initialise force terms
    Fbias = np.zeros(len(x));
    Ftot_num = np.zeros(len(x));
    Ftot_den = np.zeros(len(x));

    for i in range(total_number_of_hills):
        # Build metadynamics potential
        s = HILLS[i, 1]  # center position of gausian
        sigma_meta2 = HILLS[i, 2] ** 2  # width of gausian
        gamma = HILLS[i, 4]  #scaling factor of gausian
        height_meta = HILLS[i, 3] * ((gamma - 1) / (gamma))  # Height of Gausian
        kernelmeta = np.exp(-0.5 * (((x - s) ** 2) / (sigma_meta2)))
        Fbias = Fbias + height_meta * kernelmeta * ((x - s) / (sigma_meta2)) #Bias force due to Metadynamics potentials

        # Estimate the biased proabability density
        pb_t = np.zeros(len(x))
        Fpbt = np.zeros(len(x))
        data = position[i * stride: (i + 1) * stride] #positons of window of constant bias force.
        for j in range(stride):
            kernel = const * np.exp(- (x - data[j])**2 / (2*bw2) ) #probability density of 1 datapoint
            pb_t = pb_t + kernel #probability density of window 
            Fpbt = Fpbt + kT * kernel * (x - data[j]) / bw2

        # Estimate of the Mean Force
        Ftot_den = Ftot_den + pb_t   #total probability density
        dfds = np.divide(Fpbt, pb_t, out=np.zeros_like(Fpbt), where=pb_t != 0) + Fbias
        Ftot_num = Ftot_num + pb_t * dfds
        Ftot = np.divide(Ftot_num, Ftot_den, out=np.zeros_like(Ftot_num), where=Ftot_den != 0) #total force

        if (i+1) % (total_number_of_hills/10) == 0:
            print(str(i+1) + " / " + str(total_number_of_hills))
            
    return [Ftot_den, Ftot]

#############################################################################################

### Algorithm to run 1D MFI using a harmonic potential

def MFI_1D_hp(position, HILLS, kappa, hp_pos, bw = 0.02):
    
    #calculate some constants
    bw2 = bw**2
    stride = int(len(position) / len(HILLS[:,1]))
    const = (1 / (bw*np.sqrt(2*np.pi)*stride))
    kT = 1
    total_number_of_hills=len(HILLS[:,1])
    print("total number of hills:", total_number_of_hills)
    
    #define analytical harmonic FORCE (not energy!):
    F_harmonic = float(kappa)*(x-float(hp_pos)) 
    
    #initialise some force terms
    Fbias = np.zeros(len(x));
    Ftot_num = np.zeros(len(x));
    Ftot_den = np.zeros(len(x));

    for i in range(total_number_of_hills):
        
        # Build metadynamics potential
        s = HILLS[i, 1]  # center position of gausian
        sigma_meta2 = HILLS[i, 2] ** 2  # width of gausian
        gamma = HILLS[i, 4]  #scaling factor of gausian
        height_meta = HILLS[i, 3] * ((gamma - 1) / (gamma))  # Height of Gausian
        kernelmeta = np.exp(-0.5 * (((x - s) ** 2) / (sigma_meta2)))
        Fbias = Fbias + height_meta * kernelmeta * ((x - s) / (sigma_meta2)) #Bias force due to Metadynamics potentials

        # Estimate the biased proabability density
        pb_t = np.zeros(len(x))
        Fpbt = np.zeros(len(x))
        data = position[i * stride: (i + 1) * stride] #positons of window of constant bias force.
        for j in range(stride):
            kernel = const * np.exp(- (x - data[j])**2 / (2*bw2) ) #probability density of 1 datapoint
            pb_t = pb_t + kernel #probability density of window 
            Fpbt = Fpbt + kT * kernel * (x - data[j]) / bw2

        # Estimate of the Mean Force
        Ftot_den = Ftot_den + pb_t   #total probability density
        dfds = np.divide(Fpbt, pb_t, out=np.zeros_like(Fpbt), where=pb_t != 0) + Fbias - F_harmonic
        Ftot_num = Ftot_num + pb_t * dfds
        Ftot = np.divide(Ftot_num, Ftot_den, out=np.zeros_like(Ftot_num), where=Ftot_den != 0) #total force

#         if (i+1) % (total_number_of_hills/10) == 0:
#             print(str(i+1) + " / " + str(total_number_of_hills))
           
    return [Ftot_den, Ftot]

#############################################################################################

### Algorithm to run 2D MFI

def MFI_2D_simple(bw=0.1):   
    
    bw2 = bw**2
    # Initialize force terms
    Fbias_x = np.zeros((nbins, nbins))
    Fbias_y = np.zeros((nbins, nbins))
    Ftot_num_x = np.zeros((nbins, nbins))
    Ftot_num_y = np.zeros((nbins, nbins))
    Ftot_den = np.zeros((nbins, nbins))

    for i in range(total_number_of_hills):
        # Build metadynamics potential
        s_x = HILLS[i, 1]  # center x-position of gausian
        s_y = HILLS[i, 2]  # center y-position of gausian
        sigma_meta2_x = HILLS[i, 3] ** 2  # width of gausian
        sigma_meta2_y = HILLS[i, 4] ** 2  # width of gausian
        gamma = HILLS[i, 6]
        height_meta = HILLS[i, 5] * ((gamma - 1) / (gamma))  # Height of Gausian

        kernelmeta = np.exp(-0.5 * (((X - s_x) ** 2) / sigma_meta2_x + ((Y - s_y) ** 2) / sigma_meta2_y))  # potential erorr in calc. of s-s_t
        Fbias_x = Fbias_x + height_meta * kernelmeta * ((X - s_x) / sigma_meta2_x);  ##potential erorr in calc. of s-s_t
        Fbias_y = Fbias_y + height_meta * kernelmeta * ((Y - s_y) / sigma_meta2_y);  ##potential erorr in calc. of s-s_t

        # Biased probability density component of the force
        # Etimate the biased proabability density p_t ^ b(s)
        pb_t = np.zeros((nbins, nbins))
        Fpbt_x = np.zeros((nbins, nbins))
        Fpbt_y = np.zeros((nbins, nbins))

        data_x = position_x[i * stride: (i + 1) * stride]
        data_y = position_y[i * stride: (i + 1) * stride]
        for j in range(stride):
            kernel = const * np.exp(- ((X - data_x[j]) ** 2 + (Y - data_y[j]) ** 2) / (2 * bw2) )
            pb_t = pb_t + kernel;
            Fpbt_x = Fpbt_x + kernel * (X - data_x[j]) / bw2
            Fpbt_y = Fpbt_y + kernel * (Y - data_y[j]) / bw2

        # Calculate Mean Force
        Ftot_den = Ftot_den + pb_t;
        # Calculate x-component of Force
        dfds_x = np.divide(Fpbt_x * kT, pb_t, out=np.zeros_like(Fpbt_x), where=pb_t != 0) + Fbias_x
        Ftot_num_x = Ftot_num_x + pb_t * dfds_x
        Ftot_x = np.divide(Ftot_num_x, Ftot_den, out=np.zeros_like(Fpbt_x), where=Ftot_den != 0)
        # Calculate y-component of Force
        dfds_y = np.divide(Fpbt_y * kT, pb_t, out=np.zeros_like(Fpbt_y), where=pb_t != 0) + Fbias_y
        Ftot_num_y = Ftot_num_y + pb_t * dfds_y
        Ftot_y = np.divide(Ftot_num_y, Ftot_den, out=np.zeros_like(Fpbt_y), where=Ftot_den != 0)

        if (i+1) % (total_number_of_hills/10) == 0: 
            print(str(i+1) + " / " + str(total_number_of_hills))
            
    return [Ftot_den, Ftot_x, Ftot_y]

### Algorithm to run 2D MFI with harmonic potential

def MFI_2D_hp(kappa, ipos_x, ipos_y, bw=0.1):

    bw2 = bw**2
    
    # Initialize force terms
    Fbias_x = np.zeros((nbins, nbins))
    Fbias_y = np.zeros((nbins, nbins))
    Ftot_num_x = np.zeros((nbins, nbins))
    Ftot_num_y = np.zeros((nbins, nbins))
    Ftot_den = np.zeros((nbins, nbins))

    #Define Harmonic Force
    F_harmonic_x = float(kappa)*(X-float(ipos_x)) 
    F_harmonic_y = float(kappa)*(Y-float(ipos_y)) 


    for i in range(total_number_of_hills):
        # Build metadynamics potential
        s_x = HILLS[i, 1]  # center x-position of gausian
        s_y = HILLS[i, 2]  # center y-position of gausian
        sigma_meta2_x = HILLS[i, 3] ** 2  # width of gausian
        sigma_meta2_y = HILLS[i, 4] ** 2  # width of gausian
        gamma = HILLS[i, 6]
        height_meta = HILLS[i, 5] * ((gamma - 1) / (gamma))  # Height of Gausian

        kernelmeta = np.exp(-0.5 * (((X - s_x) ** 2) / sigma_meta2_x + ((Y - s_y) ** 2) / sigma_meta2_y))  # potential erorr in calc. of s-s_t
        Fbias_x = Fbias_x + height_meta * kernelmeta * ((X - s_x) / sigma_meta2_x);  ##potential erorr in calc. of s-s_t
        Fbias_y = Fbias_y + height_meta * kernelmeta * ((Y - s_y) / sigma_meta2_y);  ##potential erorr in calc. of s-s_t

        # Biased probability density component of the force
        # Etimate the biased proabability density p_t ^ b(s)
        pb_t = np.zeros((nbins, nbins))
        Fpbt_x = np.zeros((nbins, nbins))
        Fpbt_y = np.zeros((nbins, nbins))

        data_x = position_x[i * stride: (i + 1) * stride]
        data_y = position_y[i * stride: (i + 1) * stride]
        for j in range(stride):
            kernel = const * np.exp(- ((X - data_x[j]) ** 2 + (Y - data_y[j]) ** 2) / (2 * bw2) )
            pb_t = pb_t + kernel;
            Fpbt_x = Fpbt_x + kernel * (X - data_x[j]) / bw2
            Fpbt_y = Fpbt_y + kernel * (Y - data_y[j]) / bw2

        # Calculate Mean Force
        Ftot_den = Ftot_den + pb_t;
        # Calculate x-component of Force
        dfds_x = np.divide(Fpbt_x * kT, pb_t, out=np.zeros_like(Fpbt_x), where=pb_t != 0) + Fbias_x - F_harmonic_x
        Ftot_num_x = Ftot_num_x + pb_t * dfds_x
        Ftot_x = np.divide(Ftot_num_x, Ftot_den, out=np.zeros_like(Fpbt_x), where=Ftot_den != 0)
        # Calculate y-component of Force
        dfds_y = np.divide(Fpbt_y * kT, pb_t, out=np.zeros_like(Fpbt_y), where=pb_t != 0) + Fbias_y - F_harmonic_y
        Ftot_num_y = Ftot_num_y + pb_t * dfds_y
        Ftot_y = np.divide(Ftot_num_y, Ftot_den, out=np.zeros_like(Fpbt_y), where=Ftot_den != 0)

        if (i+1) % (total_number_of_hills/10) == 0: 
            print(str(i+1) + " / " + str(total_number_of_hills))
            
    return [Ftot_den, Ftot_x, Ftot_y]

#############################################################################################

### Patching 1D simulations 

def patch_1D(master_array):
    F = np.zeros(len(master_array[0][0]))
    F_den = np.zeros(len(master_array[0][0]))

    for i in range(len(master_array)):
        F += master_array[i][0] * master_array[i][1]
        F_den += master_array[i][0]

    F = np.divide(F, F_den, out=np.zeros_like(F), where=F_den != 0)  
    return [F_den, F]

### Patching 2D simulations 

def patch_2D(master_array):

    FX = np.zeros((nbins, nbins))
    FY = np.zeros((nbins, nbins))
    FP = np.zeros((nbins, nbins))

    for i in range(len(master)):
        FX += master_array[i][0] * master_array[i][1]
        FY += master_array[i][0] * master_array[i][2]
        FP += master_array[i][0]

    FX = np.divide(FX, FP, out=np.zeros_like(FX), where=FP != 0)
    FY = np.divide(FY, FP, out=np.zeros_like(FY), where=FP != 0)
    
    return [FP, FX, FY]

# 1D Integration

In [24]:
### Integrtion using Fast Fourier Transform (FFT integration) in 1D
### centered around 0

def FFT_intg_1D(Ftot):
    #Fourier Transform
    fhat = np.fft.fft(Ftot)
    
    #Calculate frequencyies
    kappa = np.fft.fftfreq(nbins, grid_space) 
    kappa = np.where(kappa != 0, kappa , 1E-10)
    
    
    #Integration of Fourier coefficients
    dfhat = fhat / ( 2 * np.pi * 1j * kappa)
    
    #Inverse Fourier Transform
    fes = np.real(np.fft.ifft(dfhat))

    fes = fes - np.min(fes)
    return fes ,kappa

### FFT integration in 1D not centered around 0

#

### Integration using Simpson's methods in 1D

def intg_1D(F):
    fes = []
    for j in range(len(x)): fes.append(integrate.simps(F[:j + 1], x[:j + 1]))
    fes = fes - min(fes)
    return fes

# 2D Integration

In [25]:
### Integrtion using Fast Fourier Transform (FFT integration) in 2D
### centered around 0

def FFT_intg_2D(FX, FY):

    #Calculate frequency
    freq_1d = np.fft.fftfreq(nbins, grid_space)
    freq_x, freq_y = np.meshgrid(freq_1d, freq_1d)
    freq_hypot = np.hypot(freq_x, freq_y)
    freq_sq = np.where(freq_hypot != 0, freq_hypot ** 2, 1E-10)
    #FFTransform and integration
    fourier_x = (np.fft.fft2(FX) * freq_x) / (2 * np.pi * 1j * freq_sq)
    fourier_y = (np.fft.fft2(FY) * freq_y) / (2 * np.pi * 1j * freq_sq)
    #Reverse FFT
    fes_x = np.real(np.fft.ifft2(fourier_x))
    fes_y = np.real(np.fft.ifft2(fourier_y))
    #Construct whole FES
    fes = fes_x + fes_y
    fes = fes - np.min(fes)
    return fes

### FFT integration in 2D with option to create finer gradients using interpolation, centered around 0

def FFT_intg_2D_interpolate(FX, FY, i_bins):

    grid_new_x = np.linspace(grid.min(), grid.max(), i_bins[0])
    grid_new_y = np.linspace(grid.min(), grid.max(), i_bins[1])
    X_new, Y_new = np.meshgrid(grid_new_x, grid_new_y)
    grid_space_x = (grid.max() - grid.min()) / (i_bins[0] - 1)
    grid_space_y = (grid.max() - grid.min()) / (i_bins[1] - 1)

    r = np.stack([X.ravel(), Y.ravel()]).T
    Sx = interpolate.CloughTocher2DInterpolator(r, FX.ravel())
    Sy = interpolate.CloughTocher2DInterpolator(r, FY.ravel())
    ri = np.stack([X_new.ravel(), Y_new.ravel()]).T

    FX = Sx(ri).reshape(X_new.shape)
    FY = Sy(ri).reshape(Y_new.shape)

    freq_1d_x = np.fft.fftfreq(i_bins[0], grid_space_x)
    freq_1d_y = np.fft.fftfreq(i_bins[1], grid_space_y)
    freq_x, freq_y = np.meshgrid(freq_1d_x, freq_1d_y)


    freq_hypot = np.hypot(freq_x, freq_y)
    freq_sq = np.where(freq_hypot != 0, freq_hypot ** 2, 1E-10)

    fourier_x = (np.fft.fft2(FX) * freq_x) / (2 * np.pi * 1j * freq_sq)
    fes_x = np.real(np.fft.ifft2(fourier_x))

    fourier_y = (np.fft.fft2(FY) * freq_y) / (2 * np.pi * 1j * freq_sq)
    fes_y = np.real(np.fft.ifft2(fourier_y))

    fes = fes_x + fes_y
    fes = fes - np.min(fes)

    return (X_new, Y_new, fes)


### FFT integration in 2D not centered around 0

def FFT_intg_2D_nz(FX, FY):

    freq_1d = np.fft.fftfreq(nbins, grid_space)
    freq_x, freq_y = np.meshgrid(freq_1d, freq_1d)

    freq_hypot = np.hypot(freq_x, freq_y)
    freq_sq = np.where(freq_hypot != 0, freq_hypot ** 2, 1E-10)

    fourier_x = (np.fft.fft2(FX) * freq_x) / (2 * np.pi * 1j * freq_sq)
    fes_x = np.real(np.fft.ifft2(fourier_x))

    fourier_y = (np.fft.fft2(FY) * freq_y) / (2 * np.pi * 1j * freq_sq)
    fes_y = np.real(np.fft.ifft2(fourier_y))

    fes = fes_x + fes_y
    fes = fes - np.min(fes)

    return fes

### 2D Finite difference method
### First integrating along x-axis with y=y_min, then integrating along y-axis

def intg_FD(FX, FY):
    SdZx = np.cumsum(FX, axis=1) * grid_space  # cumulative sum along x-axis
    SdZy = np.cumsum(FY, axis=0) * grid_space  # cumulative sum along y-axis
    
    FES = np.zeros((nbins,nbins))
    
    for i in range(FES.shape[0]):
        for j in range(FES.shape[1]):
            FES[i, j]  += np.sum([SdZy[i, 0], -SdZy[0, 0], SdZx[i, j], -SdZx[i, 0]])
            
    
    FES = FES - np.min(FES)
    return FES

### Extensive 2D Finite difference method
### integrating along x-axis with y=y_min, then integrating along y-axis
### then integrating along  x-axis with y=y_max, then integrating along y-axis
### and repeating all posible integration paths (8 in total) 
### and finally taking the average of all integrals.


def intg_FD8(FX,FY):

    SdZx = np.cumsum(FX, axis=1) * grid_space  # cumulative sum along x-axis
    SdZy = np.cumsum(FY, axis=0) * grid_space  # cumulative sum along y-axis
    SdZx3 = np.cumsum(FX[::-1], axis=1) * grid_space  # cumulative sum along x-axis
    SdZy3 = np.cumsum(FY[::-1], axis=0) * grid_space  # cumulative sum along y-axis
    SdZx5 = np.cumsum(FX[:, ::-1], axis=1) * grid_space  # cumulative sum along x-axis
    SdZy5 = np.cumsum(FY[:, ::-1], axis=0) * grid_space  # cumulative sum along y-axis
    SdZx7 = np.cumsum(FX[::-1, ::-1], axis=1) * grid_space  # cumulative sum along x-axis
    SdZy7 = np.cumsum(FY[::-1, ::-1], axis=0) * grid_space  # cumulative sum along y-axis


    FES = np.zeros(i_bins)
    FES2 = np.zeros(i_bins)
    FES3 = np.zeros(i_bins)
    FES4 = np.zeros(i_bins)
    FES5 = np.zeros(i_bins)
    FES6 = np.zeros(i_bins)
    FES7 = np.zeros(i_bins)
    FES8 = np.zeros(i_bins)

    for i in range(FES.shape[0]):
        for j in range(FES.shape[1]):
            FES[i, j]  += np.sum([SdZy[i, 0], -SdZy[0, 0], SdZx[i, j], -SdZx[i, 0]])
            FES2[i, j] += np.sum([SdZx[0, j], -SdZx[0, 0], SdZy[i, j], -SdZy[0, j]])
            FES3[i, j] += np.sum([-SdZy3[i, 0], SdZy3[0, 0], SdZx3[i, j], -SdZx3[i, 0]])
            FES4[i, j] += np.sum([SdZx3[0, j], -SdZx3[0, 0], -SdZy3[i, j], SdZy3[0, j]])
            FES5[i, j] += np.sum([SdZy5[i, 0], -SdZy5[0, 0], -SdZx5[i, j], SdZx5[i, 0]])
            FES6[i, j] += np.sum([-SdZx5[0, j], SdZx5[0, 0], SdZy5[i, j], -SdZy5[0, j]])
            FES7[i, j] += np.sum([-SdZy7[i, 0], SdZy7[0, 0], -SdZx7[i, j], SdZx7[i, 0]])
            FES8[i, j] += np.sum([-SdZx7[0, j], SdZx7[0, 0], -SdZy7[i, j], SdZy7[0, j]])

    FES = FES - np.min(FES)
    FES2 = FES2 - np.min(FES2)
    FES3 = FES3[::-1] - np.min(FES3)
    FES4 = FES4[::-1] - np.min(FES4)
    FES5 = FES5[:,::-1] - np.min(FES5)
    FES6 = FES6[:,::-1] - np.min(FES6)
    FES7 = FES7[::-1,::-1] - np.min(FES7)
    FES8 = FES8[::-1,::-1] - np.min(FES8)
    FES_a = (FES + FES2 + FES3 + FES4 + FES5 + FES6 + FES7 + FES8) / 8
    FES_a = FES_a - np.min(FES_a)

    return FES_a

### Extensive 2D Finite difference method with option to create finer gradients using interpolation

def intg_FD8_interpolate(FX,FY, i_bins):

    r = np.stack([X.ravel(), Y.ravel()]).T
    Sx = interpolate.CloughTocher2DInterpolator(r, FX.ravel())
    Sy = interpolate.CloughTocher2DInterpolator(r, FY.ravel())
    Nx, Ny = i_bins

    x_new = np.linspace(grid.min(), grid.max(), Nx)
    y_new = np.linspace(grid.min(), grid.max(), Ny)
    X_new, Y_new = np.meshgrid(x_new, y_new)

    ri = np.stack([X_new.ravel(), Y_new.ravel()]).T
    FX = Sx(ri).reshape(X_new.shape)
    FY = Sy(ri).reshape(Y_new.shape)

    grid_diff = np.diff(x_new)[0]


    SdZx = np.cumsum(FX, axis=1) * grid_diff  # cumulative sum along x-axis
    SdZy = np.cumsum(FY, axis=0) * grid_diff  # cumulative sum along y-axis
    SdZx3 = np.cumsum(FX[::-1], axis=1) * grid_diff  # cumulative sum along x-axis
    SdZy3 = np.cumsum(FY[::-1], axis=0) * grid_diff  # cumulative sum along y-axis
    SdZx5 = np.cumsum(FX[:, ::-1], axis=1) * grid_diff  # cumulative sum along x-axis
    SdZy5 = np.cumsum(FY[:, ::-1], axis=0) * grid_diff  # cumulative sum along y-axis
    SdZx7 = np.cumsum(FX[::-1, ::-1], axis=1) * grid_diff  # cumulative sum along x-axis
    SdZy7 = np.cumsum(FY[::-1, ::-1], axis=0) * grid_diff  # cumulative sum along y-axis


    FES = np.zeros(i_bins)
    FES2 = np.zeros(i_bins)
    FES3 = np.zeros(i_bins)
    FES4 = np.zeros(i_bins)
    FES5 = np.zeros(i_bins)
    FES6 = np.zeros(i_bins)
    FES7 = np.zeros(i_bins)
    FES8 = np.zeros(i_bins)

    for i in range(FES.shape[0]):
        for j in range(FES.shape[1]):
            FES[i, j]  += np.sum([SdZy[i, 0], -SdZy[0, 0], SdZx[i, j], -SdZx[i, 0]])
            FES2[i, j] += np.sum([SdZx[0, j], -SdZx[0, 0], SdZy[i, j], -SdZy[0, j]])
            FES3[i, j] += np.sum([-SdZy3[i, 0], SdZy3[0, 0], SdZx3[i, j], -SdZx3[i, 0]])
            FES4[i, j] += np.sum([SdZx3[0, j], -SdZx3[0, 0], -SdZy3[i, j], SdZy3[0, j]])
            FES5[i, j] += np.sum([SdZy5[i, 0], -SdZy5[0, 0], -SdZx5[i, j], SdZx5[i, 0]])
            FES6[i, j] += np.sum([-SdZx5[0, j], SdZx5[0, 0], SdZy5[i, j], -SdZy5[0, j]])
            FES7[i, j] += np.sum([-SdZy7[i, 0], SdZy7[0, 0], -SdZx7[i, j], SdZx7[i, 0]])
            FES8[i, j] += np.sum([-SdZx7[0, j], SdZx7[0, 0], -SdZy7[i, j], SdZy7[0, j]])

    FES = FES - np.min(FES)
    FES2 = FES2 - np.min(FES2)
    FES3 = FES3[::-1] - np.min(FES3)
    FES4 = FES4[::-1] - np.min(FES4)
    FES5 = FES5[:,::-1] - np.min(FES5)
    FES6 = FES6[:,::-1] - np.min(FES6)
    FES7 = FES7[::-1,::-1] - np.min(FES7)
    FES8 = FES8[::-1,::-1] - np.min(FES8)
    FES_a = (FES + FES2 + FES3 + FES4 + FES5 + FES6 + FES7 + FES8) / 8
    FES_a = FES_a - np.min(FES_a)

    return (FES_a, X_new, Y_new)

# Calculating the average deviation

In [41]:
# Calculate average deviation of 1D FES

def error_1D(FES):
    AD = abs(FES - y)
    AAD = sum(AD) / len(AD)
    print("The AAD of the FES is: " + str(AAD))
    return (AD, AAD)

# Calculate average deviation of 1D FES in central region [range_min, range_max]. e.g. [-1.75, 1.75]
def error_1D_centre(FES, range_min, range_max):
    AD = abs(FES[index(-1.75):index(1.75)+1] - y[index(-1.75):index(1.75)+1])
    AAD = sum(AD) / len(AD)
    print("The AAD of the FES from x=" + str(range_min) +" to x=" + str(range_max) + " is: " + str(AAD))
    return (AD, AAD)

# Calculate average deviation of 2D FES

def error_2D(FES):
    AD = abs(FES - Z)
    AAD = sum(sum(AD))/(nbins**2)
    print("The AAD of the FES is: " + str(AAD))
    return (AD, AAD)

def error_2D_cutoff(FES):
    Flim = 25
    AD = np.where(FES < Flim, abs(FES - Z), 0)
    AAD = sum(sum(AD))/(np.count_nonzero(AD))
    print("The AAD of the FES is: " + str(AAD))
    print(Flim)
    return (AD, AAD)



# Other Functions

In [49]:
#define indexing
def index(position):
    return int((position-min_x)//grid_space) + 1

def index_plumed(position, min_x, grid_space):
    return int((position-min_x)//grid_space)

In [45]:
# Turn every zero of an array into NaN
def zero_to_nan(input_array):
    len_x, len_y = np.shape(input_array)
    for ii in range(len_x):
        for jj in range(len_y):
            if input_array[ii][jj] == 0: input_array[ii][jj] = float("Nan")
    return input_array